# Brute force solver for Sudoku

Brute force is seldom the way to solve a puzzle for which there are well documented algorithms. However, brute force is the simplest way to solve *every* Sudoku puzzle.

When solving Sudoku puzzles algorthimicly, some puzzles can be solved with a single algorithm while others require multiple agorithms of varying degree of complexity.

### Helper functions

In [81]:
import re
import timeit
from collections import Counter
from copy import deepcopy

In [7]:
def load_puzzle(puzzle_text, dimensions=1):
    """Load a puzzle string into a 1D or 2D array of integers."""
    # replace periods with 0 if they exist
    new_text = puzzle_text.replace('.', '0')
    digits = [int(x) for x in list(new_text)]
    if dimensions == 1:
        return digits
    else:
        arr = []
        for _ in range(9):
            subset, digits = digits[0:9], digits[9:]
            arr.append(subset)
        return arr

In [56]:
def pprint_single_board(puzzle):
    """Pretty print a single Sudoku board with borders. Automatically detect
    if the puzzle is 1D or 2D."""

    # convert to a 1D array if necessary
    if len(puzzle) > 80:    # 1D array
        my_puz = puzzle.copy()
    else:                   # 2D array
        my_puz = [puzzle[x][y] for x in range(9) for y in range(9)]

    # replace zeroes with spaces
    for idx in range(81):
        my_puz[idx] = " " if my_puz[idx] == 0 else my_puz[idx]

    print(F'+{"-"*7}+{"-"*7}+{"-"*7}+')
    for ids in range(3):
        for idr in range(3):
            row = ids*3 + idr
            start = row * 9
            print(F'| {my_puz[start]} {my_puz[start+1]} {my_puz[start+2]} ', end='')
            print(F'| {my_puz[start+3]} {my_puz[start+4]} {my_puz[start+5]} ', end='')
            print(F'| {my_puz[start+6]} {my_puz[start+7]} {my_puz[start+8]} |')

        print(F'+{"-"*7}+{"-"*7}+{"-"*7}+')

def pprint_dual_boards(puzzle1, puzzle2):
    """Pretty print two Sudoku boards with border side by side. Automatically detect
    if the puzzle is 1D or 2D."""

    # convert to a 1D array if necessary
    if len(puzzle1) > 80:   # 1D array
        arr1 = puzzle1.copy()
        arr2 = puzzle2.copy()
    else:
        arr1 = [puzzle1[x][y] for x in range(9) for y in range(9)]
        arr2 = [puzzle2[x][y] for x in range(9) for y in range(9)]

    for idx in range(81):
        arr1[idx] = " " if arr1[idx] == 0 else arr1[idx]
        arr2[idx] = " " if arr2[idx] == 0 else arr2[idx]

    print(F'+{"-"*7}+{"-"*7}+{"-"*7}+  +{"-"*7}+{"-"*7}+{"-"*7}+')
    for ids in range(3):
        for idr in range(3):
            row = ids*3 + idr
            start = row * 9
            print(F'| {arr1[start]} {arr1[start+1]} {arr1[start+2]} ', end='')
            print(F'| {arr1[start+3]} {arr1[start+4]} {arr1[start+5]} ', end='')
            print(F'| {arr1[start+6]} {arr1[start+7]} {arr1[start+8]} |', end='  ')
            print(F'| {arr2[start]} {arr2[start+1]} {arr2[start+2]} ', end='')
            print(F'| {arr2[start+3]} {arr2[start+4]} {arr2[start+5]} ', end='')
            print(F'| {arr2[start+6]} {arr2[start+7]} {arr2[start+8]} |')

        print(F'+{"-"*7}+{"-"*7}+{"-"*7}+  +{"-"*7}+{"-"*7}+{"-"*7}+')

## My Brute Force Solver

In [10]:
def get_valid_value(puz,pos,val):
    # check this row
    row = pos // 9  # integer division determines the row
    if val in puz[9*row:9*row+9]:
        return get_valid_value(puz,pos,val+1)

    # check this column
    col = pos % 9   # remainder determines column
    column_contents = [puz[idy*9+col] for idy in range(9)]
    if val in column_contents:
        return get_valid_value(puz,pos,val+1)

    # check this section; determining the range of values that make up a 3x3 section
    # is more complex that determining the row or column
    section_start = (row - row % 3) * 9 + (col // 3) * 3
    section_contents = [puz[section_start+idc+idr*9] for idc in range(3) for idr in range(3)]
    if val in section_contents:
        return get_valid_value(puz,pos,val+1)

    return val

def find_empty(puz):
    """Find the first empty location in the puzzle and return None if there are no empties."""
    for idx in range(81):
        if puz[idx] == 0:
            return idx
    return None

def brute_force_sudoku(puzzle):

    # start at first empty cell
    position = find_empty(puzzle)
    # history is empty because we haven't solved any cells
    history = []

    while 0 in puzzle:

        # retrieve a new value for the current position that is at least one greater than
        # the current value and does not create any duplicate values
        value = get_valid_value(puzzle,position,puzzle[position]+1)

        # if the current value is greater than 9, then we have run out of guesses
        # that could work and one of our previous guesses is actually invalid;
        # we must "erase" this current position and go back to the last position
        # that we set
        if value > 9:
            puzzle[position] = 0
            position = history.pop()    # go back

        # if the new value is 9 or less, then store it, save the current location
        # in history, and find the next empty cell
        else:
            puzzle[position] = value
            history.append(position)
            position = find_empty(puzzle)

## Version 2 of my algorithm

Convert `while` loop to only look forward in the puzzle and not start from the beginning.

In [11]:
def get_valid_value_v2(puz,pos,val):
    # check this row
    row = pos // 9  # integer division determines the row
    if val in puz[9*row:9*row+9]:
        return get_valid_value(puz,pos,val+1)

    # check this column
    col = pos % 9   # remainder determines column
    column_contents = [puz[idy*9+col] for idy in range(9)]
    if val in column_contents:
        return get_valid_value(puz,pos,val+1)

    # check this section; determining the range of values that make up a 3x3 section
    # is more complex that determining the row or column
    section_start = (row - row % 3) * 9 + (col // 3) * 3
    section_contents = [puz[section_start+idc+idr*9] for idc in range(3) for idr in range(3)]
    if val in section_contents:
        return get_valid_value(puz,pos,val+1)

    return val

def find_empty_v2(puz,pos):
    """Find the first empty location in the puzzle and return None if there are no empties."""
    for idx in range(pos,81):
        if puz[idx] == 0:
            return idx
    return None

def brute_force_sudoku_v2(puzzle):

    # start at first empty cell
    position = find_empty_v2(puzzle,0)
    # history is empty because we haven't solved any cells
    history = []

    while position is not None:

        # retrieve a new value for the current position that is at least one greater than
        # the current value and does not create any duplicate values
        value = get_valid_value_v2(puzzle,position,puzzle[position]+1)

        # if the current value is greater than 9, then we have run out of guesses
        # that could work and one of our previous guesses is actually invalid;
        # we must "erase" this current position and go back to the last position
        # that we set
        if value > 9:
            puzzle[position] = 0
            position = history.pop()    # go back

        # if the new value is 9 or less, then store it, save the current location
        # in history, and find the next empty cell
        else:
            puzzle[position] = value
            history.append(position)
            position = find_empty_v2(puzzle,position+1)

## Version 3 of my algorithm

Remove list comprehension from `get_valid_value`. Instead of building the column contents into a separate array (which takes time to create that variable), walk through the existing puzzle array.

In [29]:
def get_valid_value_v3(puz,pos,val):

    row = (pos // 9) * 9    # integer division determines the row
    col = pos % 9           # remainder determines column
    for idx in range(9):
        if puz[row+idx] == val or puz[9*idx+col] == val:
            return get_valid_value_v3(puz,pos,val+1)
        
    section_start = (row // 27) * 27 + (col // 3) * 3
    for idx in [0,9,18]:
        for idy in range(3):
            cursor = section_start + idx + idy
            if puz[cursor] == val:
                return get_valid_value_v3(puz,pos,val+1)
            
    return val


def brute_force_sudoku_v3(puzzle):

    # start at first empty cell
    position = find_empty_v2(puzzle,0)
    # history is empty because we haven't solved any cells
    history = []

    while position is not None:

        # retrieve a new value for the current position that is at least one greater than
        # the current value and does not create any duplicate values
        value = get_valid_value_v3(puzzle,position,puzzle[position]+1)

        # if the current value is greater than 9, then we have run out of guesses
        # that could work and one of our previous guesses is actually invalid;
        # we must "erase" this current position and go back to the last position
        # that we set
        if value > 9:
            puzzle[position] = 0
            position = history.pop()    # go back

        # if the new value is 9 or less, then store it, save the current location
        # in history, and find the next empty cell
        else:
            puzzle[position] = value
            history.append(position)
            position = find_empty_v2(puzzle,position+1)


## Version 4 of my solver

Convert to 2D array.

In [78]:
def get_valid_value_v4(puz,row,col,val):
    
    # check this row and column
    for idx in range(9):
        if puz[idx][col] == val or puz[row][idx] == val:
            return get_valid_value_v4(puz,row,col,val+1)

    # check this section; determining the range of values that make up a 3x3 section
    # is more complex that determining the row or column
    section_row = row - row % 3
    section_col = col - col % 3
    for idx in range(3):
        for idy in range(3):
            if puz[section_row + idx][section_col + idy] == val:
                return get_valid_value_v4(puz,row,col,val+1)

    return val

def find_empty_v4(puz,row,col):
    """Find the first empty location in the puzzle and return None if there are no empties."""
    for idx in range(row,9):
        for idy in range(9):
            if puz[idx][idy] == 0:
                return idx,idy
    return None,None

def brute_force_sudoku_v4(puzzle):

    # start at first empty cell
    row,col = find_empty_v4(puzzle,0,0)
    # history is empty because we haven't solved any cells
    history = []

    while row is not None:

        # retrieve a new value for the current position that is at least one greater than
        # the current value and does not create any duplicate values
        value = get_valid_value_v4(puzzle,row,col,puzzle[row][col]+1)

        # if the current value is greater than 9, then we have run out of guesses
        # that could work and one of our previous guesses is actually invalid;
        # we must "erase" this current position and go back to the last position
        # that we set
        if value > 9:
            puzzle[row][col] = 0
            try:
                row,col = history.pop()    # go back
            except:
                pprint_single_board(puzzle)
                return None


        # if the new value is 9 or less, then store it, save the current location
        # in history, and find the next empty cell
        else:
            puzzle[row][col] = value
            history.append((row,col))
            row,col = find_empty_v4(puzzle,row,col)

## ChatGPT Brute Force Solver

In [88]:
def is_valid(board, row, col, num):
    # Check if 'num' can be placed on Sudoku board at board[row][col]
    block_row, block_col = 3 * (row // 3), 3 * (col // 3)

    for i in range(9):
        if board[row][i] == num or board[i][col] == num:
            return False
        if board[block_row + i // 3][block_col + i % 3] == num:
            return False
    return True

def solve_sudoku(board):
    empty = find_empty_location(board)
    if not empty:
        return True  # Puzzle solved
    row, col = empty

    for num in range(1, 10):  # Numbers 1 to 9
        if is_valid(board, row, col, num):
            board[row][col] = num
            if solve_sudoku(board):
                return True
            board[row][col] = 0  # Reset if moving forward isn't possible

    return False  # Trigger backtracking

def find_empty_location(board):
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:  # Empty cells are represented by 0
                return i, j
    return None

def print_board(board):
    for row in board:
        print(" ".join(str(num) if num != 0 else '.' for num in row))


Solve a challenging puzzle with my best algorithm and ChatGPT's algorithm and make sure the solutions match.

In [89]:
# beyond nightmare
beyond_nightmare = '12.3.....4.....3....3.5......42..5......8...9.6...5.7...15..2......9..6......7..8'
original = load_puzzle(beyond_nightmare, dimensions=2)

my_brute = deepcopy(original)
brute_force_sudoku_v4(my_brute)

gpt_brute = deepcopy(original)
solve_sudoku(gpt_brute)
pprint_dual_boards(my_brute, gpt_brute)

if my_brute == gpt_brute:
    print("Identical solutions!")
else:
    print("Error: the solutions do not match.")


+-------+-------+-------+  +-------+-------+-------+
| 1 2 5 | 3 7 4 | 8 9 6 |  | 1 2 5 | 3 7 4 | 8 9 6 |
| 4 7 9 | 6 1 8 | 3 2 5 |  | 4 7 9 | 6 1 8 | 3 2 5 |
| 6 8 3 | 9 5 2 | 7 1 4 |  | 6 8 3 | 9 5 2 | 7 1 4 |
+-------+-------+-------+  +-------+-------+-------+
| 7 1 4 | 2 6 9 | 5 8 3 |  | 7 1 4 | 2 6 9 | 5 8 3 |
| 5 3 2 | 7 8 1 | 6 4 9 |  | 5 3 2 | 7 8 1 | 6 4 9 |
| 9 6 8 | 4 3 5 | 1 7 2 |  | 9 6 8 | 4 3 5 | 1 7 2 |
+-------+-------+-------+  +-------+-------+-------+
| 8 9 1 | 5 4 6 | 2 3 7 |  | 8 9 1 | 5 4 6 | 2 3 7 |
| 2 5 7 | 8 9 3 | 4 6 1 |  | 2 5 7 | 8 9 3 | 4 6 1 |
| 3 4 6 | 1 2 7 | 9 5 8 |  | 3 4 6 | 1 2 7 | 9 5 8 |
+-------+-------+-------+  +-------+-------+-------+
Identical solutions!


In [91]:
%%timeit

brute_force_sudoku(load_puzzle(beyond_nightmare))

7.57 s ± 44.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [92]:
%%timeit

brute_force_sudoku_v2(load_puzzle(beyond_nightmare))

6.66 s ± 88.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [93]:
%%timeit

brute_force_sudoku_v3(load_puzzle(beyond_nightmare))

5.85 s ± 52.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [94]:
%%timeit

brute_force_sudoku_v4(load_puzzle(beyond_nightmare, dimensions=2))

5.55 s ± 82 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit

solve_sudoku(load_puzzle(beyond_nightmare, dimensions=2))

4.73 s ± 18.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Load a bank of puzzles

In [41]:
PUZZLE_BANK_SIZE = 20

puzzle_bank = []

counter = 0

with open('diabolical.txt', 'r') as file:
    for line in file:
        # split the line by whitespace
        row = re.split(r'\s+', line.strip())

        if counter < PUZZLE_BANK_SIZE:
            puzzle_bank.append(dict(name=row[0], board=row[1], diff=float(row[2])))

        counter += 1


print(len(puzzle_bank))

20


In [42]:
# test performance of my v1 code on the puzzle bank

my_setup = '''
board = load_puzzle(puzzle['board'])
original = board.copy()
'''

my_code = '''
brute_force_sudoku(board)
'''

for puzzle in puzzle_bank:
    result = timeit.timeit(setup=my_setup,stmt=my_code,number=1000,globals=globals())
    puzzle['v1 result'] = result

In [43]:
# test performance of my v4 code on the puzzle bank

my_setup = '''
board = load_puzzle(puzzle['board'],dimensions=2)
original = board.copy()
'''

my_code = '''
brute_force_sudoku_v4(board)
'''

for puzzle in puzzle_bank:
    result = timeit.timeit(setup=my_setup,stmt=my_code,number=1000,globals=globals())
    puzzle['v4 result'] = result

In [44]:
# test chatgpt performance on the puzzle bank

chatgpt_setup = '''
board = load_puzzle(puzzle['board'],dimensions=2)
original = board.copy()
'''

chatgpt_code = '''
solve_sudoku(board)
'''

for puzzle in puzzle_bank:
    result = timeit.timeit(setup=chatgpt_setup,stmt=chatgpt_code,number=1000,globals=globals())
    puzzle['chatgpt result'] = result

In [96]:
# compare performance of chatgpt and my code

min, max = 1.0, 0.0
minpuz, maxpuz = '', ''

for puzzle in puzzle_bank:
    percent = puzzle['chatgpt result'] / puzzle['v4 result']
    if percent < min:
        min = percent
        minpuz = puzzle['board']
    if percent > max:
        max = percent
        maxpuz = puzzle['board']

print(min,max)
print(F'min = {minpuz}')
print(F'max = {maxpuz}')

0.8802197269511389 1.0992093901229296
min = 083020090000800100029300008000098700070000060006740000300006980002005000010030540
max = 502070430000400050000020700400502073000010000970603001008050000010004000059030807


# Compare my algorith to ChatGPT for all puzzles

This will take a long time to run (days) because there are more than 12,000 puzzles that must be solved by three different algorithms.

In [97]:
import pandas as pd

In [98]:
my_v1_setup = '''
board = load_puzzle(puzzle)
'''

my_v1_code = '''
brute_force_sudoku(board)
'''

chatgpt_setup = '''
board = load_puzzle(puzzle,dimensions=2)
'''

chatgpt_code = '''
solve_sudoku(board)
'''

my_v4_setup = '''
board = load_puzzle(puzzle,dimensions=2)
'''

my_v4_code = '''
brute_force_sudoku_v4(board)
'''

massive = []

with open('diabolical.txt', 'r') as file:
    for line in file:
        # split the line by whitespace
        name, puzzle, diff = re.split(r'\s+', line.strip())

        # board = load_puzzle(puzzle)
        result1 = timeit.timeit(setup=my_v1_setup,stmt=my_v1_code,number=10,globals=globals())

        result3 = timeit.timeit(setup=my_v4_setup,stmt=my_v4_code,number=10,globals=globals())

        # board = load_puzzle(puzzle,dimensions=2)
        result2 = timeit.timeit(setup=chatgpt_setup,stmt=chatgpt_code,number=10,globals=globals())

        my_dict = dict(name=name, diff=diff, v1_perf=result1, v4_perf=result3, gpt_perf=result2)

        massive.append(my_dict)

In [99]:
# the above cell took 2476 minutes on the laptop for diabolical
# the above cell took 2238 minutes on the studio for diabolical
df = pd.DataFrame(massive)
df.head()

,name,diff,v1_perf,v4_perf,gpt_perf
0,00015097c6c3,7.2,0.080690,0.048227,0.053308
1,0001d2888928,7.1,0.146445,0.105099,0.110633
2,000274921f39,7.1,0.004675,0.003519,0.003764
3,00035f2db993,8.2,0.066116,0.050140,0.052692
4,0003c85d91eb,6.6,4.114708,2.928728,3.417428


In [100]:
df.to_csv('diabolical_mpbm1_v4.csv',index=False)

The following summary compares v1 and v4 performance to ChatGPT performance for every puzzle difficulty. Sudoku puzzles in this bank have a difficulty rating shown in the `diff` column. This averages the performance of all puzzles with the same rating.

The columns `v1` and `v4` show GPT performance relative to the v1 and v4 algorithms. A number less than 100 means the GPT algorithm is faster. For examle, `75.2` means that GPT solved puzzles 75.2% faster than the given algorithm. Numbers greater than 100 mean the GPT algorithm is slower. For example, `103.6` means that GPT solved puzzles in 103.6% the time (longer) of the given algorithm.

In [101]:
summary_df = df[['diff', 'v1_perf', 'v4_perf', 'gpt_perf']].groupby('diff').mean()

summary_df['v1'] = summary_df['gpt_perf']/summary_df['v1_perf'] * 100
summary_df['v4'] = summary_df['gpt_perf']/summary_df['v4_perf'] * 100

summary_df

,v1_perf,v4_perf,gpt_perf,v1,v4
diff,,,,,
5.0,0.262400,0.190169,0.197335,75.203822,103.768467
5.2,0.170236,0.125333,0.129049,75.806259,102.965125
5.4,0.180870,0.131662,0.136478,75.456204,103.657582
5.5,0.151052,0.110592,0.114620,75.881381,103.642552
5.6,0.166765,0.121998,0.126254,75.707655,103.488169
5.7,0.183348,0.133968,0.138981,75.802088,103.742016
5.8,0.409823,0.297483,0.306101,74.691173,102.897129
5.9,0.179849,0.130644,0.133482,74.219141,102.172166
6.0,0.174657,0.127896,0.130821,74.901561,102.286806
